In [25]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

# -- 1. ЗАГРУЗКА ДАННЫХ --
print("Ячейка 1: Загрузка 'чистых' данных...")
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

# Сохраняем PassengerId для файла отправки
test_passenger_ids = test_df['PassengerId']

# ВАЖНО: Мы НЕ объединяем их здесь. 
# Мы будем обрабатывать train и test отдельно, но ОДИНАКОВО.
print(f"Размер Train: {train_df.shape}")
print(f"Размер Test: {test_df.shape}")

Ячейка 1: Загрузка 'чистых' данных...
Размер Train: (891, 12)
Размер Test: (418, 11)


In [17]:
# --- ГИПОТЕЗА 2: Фича "FamilySize" (Размер Семьи) ---
print("Шаг 1: Создание 'FamilySize'...")

# Мы просто складываем колонки, как обычные числа
train_df['FamilySize'] = train_df['SibSp'] + train_df['Parch'] + 1
test_df['FamilySize'] = test_df['SibSp'] + test_df['Parch'] + 1

# --- Проверяем, что получилось ---
print("--- Распределение FamilySize ---")
print(train_df['FamilySize'].value_counts())

# --- ГИПОТЕЗА 3: Фича "IsAlone" (Одиночка) ---
print("\nШаг 2: Создание 'IsAlone'...")

# "Рецепт" в 1 строку:
# 1. (train_df['FamilySize'] == 1) - эта часть создает колонку из True/False
# 2. .astype(int) - эта часть превращает True в 1 и False в 0
train_df['IsAlone'] = (train_df['FamilySize'] == 1).astype(int)
test_df['IsAlone'] = (test_df['FamilySize'] == 1).astype(int)

# --- Проверяем, что получилось ---
print("--- Распределение IsAlone (1 = Одинок, 0 = Есть семья) ---")
print(train_df['IsAlone'].value_counts())



Шаг 1: Создание 'FamilySize'...
--- Распределение FamilySize ---
FamilySize
1     537
2     161
3     102
4      29
6      22
5      15
7      12
11      7
8       6
Name: count, dtype: int64

Шаг 2: Создание 'IsAlone'...
--- Распределение IsAlone (1 = Одинок, 0 = Есть семья) ---
IsAlone
1    537
0    354
Name: count, dtype: int64


In [28]:
# --- ЕДИНЫЙ БЛОК ПРЕДОБРАБОТКИ (Гипотезы 1-5) ---
print("\nЯчейка 2: Запуск ЕДИНОГО блока предобработки...")

# -- Создаем копии, чтобы избежать "SettingWithCopyWarning"
# Мы работаем с train_data и test_data
train_data = train_df.copy()
test_data = test_df.copy()

# --- ГИПОТЕЗА 1: Фича "Title" (Титул) ---
print("Создание 'Title'...")
# Шаг 1: Извлечение
train_data['Title'] = train_data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
test_data['Title'] = test_data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
# Шаг 2: Очистка
train_data['Title'] = train_data['Title'].replace(['Mlle', 'Ms'], 'Miss')
train_data['Title'] = train_data['Title'].replace('Mme', 'Mrs')
test_data['Title'] = test_data['Title'].replace(['Mlle', 'Ms'], 'Miss')
test_data['Title'] = test_data['Title'].replace('Mme', 'Mrs')
rare_titles = ['Dr', 'Rev', 'Col', 'Major', 'Don', 'Lady', 'Sir', 'Capt', 'Countess', 'Jonkheer', 'Dona']
train_data['Title'] = train_data['Title'].replace(rare_titles, 'Rare')
test_data['Title'] = test_data['Title'].replace(rare_titles, 'Rare')
# Заполняем 1 пропуск в test_data (если вдруг появился)
test_data['Title'] = test_data['Title'].fillna('Rare')


# --- ГИПОТЕЗА 2 и 3: "FamilySize" и "IsAlone" ---
print("Создание 'FamilySize' и 'IsAlone'...")
train_data['FamilySize'] = train_data['SibSp'] + train_data['Parch'] + 1
test_data['FamilySize'] = test_data['SibSp'] + test_data['Parch'] + 1
train_data['IsAlone'] = (train_data['FamilySize'] == 1).astype(int)
test_data['IsAlone'] = (test_data['FamilySize'] == 1).astype(int)

# --- ГИПОТЕЗА 4: "Deck" (Палуба) ---
print("Создание 'Deck'...")
train_data['Deck'] = train_data['Cabin'].fillna('U').str[0]
test_data['Deck'] = test_data['Cabin'].fillna('U').str[0]

# --- ГИПОТЕЗА 5: Заполнение пропусков (Age, Fare, Embarked) ---
print("Заполнение пропусков в 'Age', 'Fare', 'Embarked'...")
train_data['Age'] = train_data['Age'].fillna(train_data['Age'].median())
test_data['Age'] = test_data['Age'].fillna(test_data['Age'].median())
test_data['Fare'] = test_data['Fare'].fillna(test_data['Fare'].median())
train_data['Embarked'] = train_data['Embarked'].fillna(train_data['Embarked'].mode()[0])

# --- УДАЛЕНИЕ "МУСОРНЫХ" КОЛОНОК ---
# Мы удаляем их ДО кодирования
drop_cols = ['Name', 'Ticket', 'Cabin', 'PassengerId']
train_data = train_data.drop(columns=drop_cols)
test_data = test_data.drop(columns=drop_cols)

# ---  Кодирование (One-Hot Encoding) ---
print("Финальное кодирование (get_dummies)...")
categorical_features = ['Title', 'Deck', 'Sex', 'Embarked']
# Мы используем pd.concat, чтобы "взорвать" ОБА датасета ОДИНАКОВО
full_data = pd.concat([train_data, test_data], axis=0)
full_data_dummies = pd.get_dummies(full_data, columns=categorical_features, drop_first=True)

# Разделяем обратно
X_train = full_data_dummies.iloc[:len(train_data)]
X_test = full_data_dummies.iloc[len(train_data):]
y_train = train_data['Survived'] # Берем 'Survived' из train_data (до "взрыва")

# УДАЛЯЕМ 'Survived' из X_train
X_train = X_train.drop('Survived', axis=1)
X_test = X_test.drop('Survived', axis=1)

print("\n--- ПРЕДОБРАБОТКА ЗАВЕРШЕНА ---")
print("Колонки в X_train:", X_train.shape)
print("Колонки в X_test:", X_test.shape)
print("Колонки в y_train:", y_train.shape)


Ячейка 2: Запуск ЕДИНОГО блока предобработки...
Создание 'Title'...
Создание 'FamilySize' и 'IsAlone'...
Создание 'Deck'...
Заполнение пропусков в 'Age', 'Fare', 'Embarked'...
Финальное кодирование (get_dummies)...

--- ПРЕДОБРАБОТКА ЗАВЕРШЕНА ---
Колонки в X_train: (891, 22)
Колонки в X_test: (418, 22)
Колонки в y_train: (891,)


In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

# --- Ячейка 3: Обучение Модели и Отправка ---
print("\nЯчейка 3: Запуск обучения...")

# --- ШАГ 1: Масштабирование (StandardScaler) ---
# Мы "кормим" StandardScaler нашими "умными" фичами
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Данные отмасштабированы (StandardScaler).")

# --- ШАГ 2: Обучение "Чемпионской" Модели ---
# Мы используем RandomForest (который дал 78.5%)
# И "лечим" переобучение с помощью max_depth=5
model = RandomForestClassifier(
    n_estimators=1000,
    max_depth=5,            
    min_samples_leaf=3,
    random_state=42
)
model.fit(X_train_scaled, y_train)
print("Модель RandomForest (ЧЕМПИОН) ОБУЧЕНА!")

# --- ШАГ 3: Создание файла для Kaggle ---
predictions = model.predict(X_test_scaled).astype(int)

submission = pd.DataFrame({
    'PassengerId': test_passenger_ids,
    'Survived': predictions
})

# Сохраняем финальный файл
submission.to_csv('submission_FINAL_789.csv', index=False)
print("\n--- ФИНАЛЬНЫЙ ФАЙЛ 'submission_FINAL_789.csv' СОЗДАН! ---")
submission.head()


Ячейка 3: Запуск обучения...
Данные отмасштабированы (StandardScaler).
Модель RandomForest (ЧЕМПИОН) ОБУЧЕНА!

--- ФИНАЛЬНЫЙ ФАЙЛ 'submission_FINAL_789.csv' СОЗДАН! ---


,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
